In [1]:
import os
import math
import pandas as pd
from PIL import Image
import pickle
import random
import warnings
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.datasets as dset
import torchvision.transforms as T
from torchvision import models
from torch.utils.data import DataLoader, sampler, random_split
warnings.filterwarnings("ignore")

In [2]:
def fix_random_seed(seed_no=0):
    torch.manual_seed(seed_no)
    torch.cuda.manual_seed(seed_no)
    random.seed(seed_no)

In [3]:
# read the guitar info
df = pd.read_csv("D:/Study Abroad/course/CSE498/project/guitar_info.csv")

In [4]:
# load the valid guitar image indice
train_data_id = []
file_path = "D:/Study Abroad/course/CSE498/project/guitar_resized_images"

# walk through all guitar images
for root, dirs, files in os.walk(file_path):
    for file in files:
        train_data_id.append(int(file.replace(".png","")))

In [5]:
# select guitar info for valid images
df = df[df['index'].isin(train_data_id)]
df['price'] = df['price'].apply(lambda x:math.ceil(float(x.replace("$","").replace(",",""))))

In [6]:
df

,index,image,title,location,price,condition
2,119748013,https://media.guitarcenter.com/is/image/MMGS7/...,Used Carvin AC375 Acoustic Electric Guitar,"N. Olmsted, OH",925,Excellent
3,119744391,https://media.guitarcenter.com/is/image/MMGS7/...,Used Martin GPC16E Acoustic Electric Guitar,"Danbury, CT",1640,Excellent
4,119746162,https://media.guitarcenter.com/is/image/MMGS7/...,Used Gibson JERRY CANTRELL ATONE SONGWRITER Ac...,"N. Olmsted, OH",3500,Excellent
5,119739399,https://media.guitarcenter.com/is/image/MMGS7/...,Used Taylor 214CE Deluxe Acoustic Electric Guitar,"Raleigh, NC",1300,Excellent
7,119744853,https://media.guitarcenter.com/is/image/MMGS7/...,Used PRS Angelus Standard SE Acoustic Guitar,"Allen Park, MI",575,Excellent
...,...,...,...,...,...,...
4194,119740084,https://media.guitarcenter.com/is/image/MMGS7/...,Used Taylor 514CE Acoustic Electric Guitar,"Allentown, PA",1500,Great
4195,119742274,https://media.guitarcenter.com/is/image/MMGS7/...,Used Mitchell Mx430qab Acoustic Electric Guitar,"Manchester, CT",270,Great
4196,119744806,https://media.guitarcenter.com/is/image/MMGS7/...,Used Gibson J45 Standard Acoustic Electric Guitar,"Bridgeton, MO",2300,Great
4197,119739960,https://media.guitarcenter.com/is/image/MMGS7/...,Used Yamaha FG-280 Acoustic Guitar,"Monroeville, PA",400,Great


In [7]:
# calculate the mean and std of R, G, and B channels for my guitar image data sets
path = "D:/Study Abroad/course/CSE498/project/guitar_resized_images"

# define a tensor transformer
image_transformer = T.ToTensor()

def normal_para(path, transformer):
    tensors = []
    # walk through all images and convert them to tensors
    for index in df['index'].values:
        file_name = '000000'+str(index) + '.png'
        file_path = os.path.join(path,file_name)
        image = Image.open(file_path)
        tensor = transformer(image)
        tensors.append(tensor)

    # stack all tensors into a single tensor
    tensor_stack = torch.stack(tensors, dim=0)

    # calculate the mean and std
    mean = torch.mean(tensor_stack, dim=[0, 2, 3])
    std = torch.std(tensor_stack, dim=[0, 2, 3])
    
    return mean,std

# calculate the mean and std of R, G, and B channels
mean,std = normal_para(path, image_transformer)

In [8]:
# define a class to generate training, validation, and testing set
class Guitar_Image_Data_Set:
    """ this class is designed to generate training, validation, and testing set from guitar images"""
    def __init__(self,dataframe,path,transform):
        self.df = dataframe
        self.path = path
        self.tf = transform
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,idx):
        file_name = '000000'+ str(df.iloc[idx]['index']) + '.png'
        file_path = os.path.join(self.path,file_name)
        image = Image.open(file_path)
        image = self.tf(image)
        price = df.iloc[idx]['price']
        
        return image, price

In [9]:
# transform images to tensors and then normalize tensors
transform = T.Compose([
            T.ToTensor(),
            T.Normalize(mean, std)
            ])

# create an instance of the Dataset
guitar_dataset = Guitar_Image_Data_Set(dataframe = df,
                                       path = path,
                                       transform = transform)

# define the ratio for training, validation, and testing set
total = len(guitar_dataset)
train = 0.7
validation = 0.15

# split the whole dataset into training, validation, and testing set
train_size = int(train * total)
validation_size = int(validation * total)
test_size = total - train_size - validation_size
train_dataset, validation_dataset, test_dataset = random_split(guitar_dataset, [train_size, validation_size, test_size])

In [10]:
# Use DataLoader to batch and shuffle the dataset
loader_train = DataLoader(train_dataset, batch_size=64,shuffle = True)
loader_val = DataLoader(validation_dataset, batch_size=64,shuffle = True)
loader_test = DataLoader(test_dataset, batch_size=64,shuffle = True)

In [11]:
# define the configuration of torch
dtype = torch.float

gpu_index = torch.randint(0, torch.cuda.device_count(), (1,)).item()
device = torch.device('cuda:{}'.format(gpu_index))
print("PyTorch is using GPU {}!".format(device))

# Constant to control how frequently we print train loss
print_every = 100

PyTorch is using GPU cuda:0!


In [12]:
# define a function that returns a consistent, predetermined random number
def fix_random_seed(seed_no=0):
    torch.manual_seed(seed_no)
    torch.cuda.manual_seed(seed_no)
    random.seed(seed_no)

# define functions for adjusting learning rate
def adjust_learning_rate(optimizer, lrd, epoch, schedule):
    """
    Multiply lrd to the learning rate if epoch is in schedule

    Inputs:
    - optimizer: An Optimizer object we will use to train the model
    - lrd: learning rate decay; a factor multiplied at scheduled epochs
    - epochs: the current epoch number
    - schedule: the list of epochs that requires learning rate update

    Returns: Nothing, but learning rate might be updated
    """
    if epoch in schedule:
        for param_group in optimizer.param_groups:
            print('lr decay from {} to {}'.format(param_group['lr'], param_group['lr'] * lrd))
            param_group['lr'] *= lrd

# define functions for training the model
def train_model(model, optimizer, epochs=1, learning_rate_decay=.1, schedule=[], criterion = nn.MSELoss()):
    """
    Train a model on guitar image dataset using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: Nothing, but prints model accuracies during training.
    """
    
    num_iters = epochs * len(loader_train)
    num_prints = num_iters // print_every + 1

    for e in range(epochs):
        
        adjust_learning_rate(optimizer, learning_rate_decay, e, schedule)

        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=dtype)
            outputs = model(x)
            
            # calculatet the loss using assigned criterion 
            loss = criterion(outputs, y.view(-1, 1))
            
            # Zero out all of the gradients for the variables which the optimizer will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            tt = t + e * len(loader_train)
            print('Epoch %d, Iteration %d, loss = %.0f' % (e, tt, loss.item()))

In [13]:
# train a toy model - a neural network with two fully connected layers
fix_random_seed(0)

C, H, W = 3, 200, 200

# initialize the output layer with 1 node for regression tasks.
output_size = 1 

hidden_layer_size = 4096
learning_rate = 1e-6
weight_decay = 1e-3
momentum = 0.5
epochs = 5

# To give a specific name to each module, use OrderedDict.
model = nn.Sequential(OrderedDict([
  ('flatten', nn.Flatten()),
  ('fc1', nn.Linear(C * H * W, hidden_layer_size)),
  ('relu1', nn.ReLU()),
  ('fc2', nn.Linear(hidden_layer_size, output_size)),
]))

model.to(device)

# use SGD as the optimizer
optimizer = optim.SGD(model.parameters(), 
                      lr=learning_rate,
                      weight_decay=weight_decay,
                      momentum=momentum,
                      nesterov=True)

# print the Architecture of the model
print('Architecture:')
print(model)
print()

# print the loss in the training process
print('Training:')
train_model(model, optimizer,epochs = epochs)

Architecture:
Sequential(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=120000, out_features=4096, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=4096, out_features=1, bias=True)
)

Training:
Epoch 0, Iteration 0, loss = 2932893
Epoch 0, Iteration 1, loss = 1861908
Epoch 0, Iteration 2, loss = 3294296
Epoch 0, Iteration 3, loss = 1470952
Epoch 0, Iteration 4, loss = 1024124
Epoch 0, Iteration 5, loss = 1872902
Epoch 0, Iteration 6, loss = 590646
Epoch 0, Iteration 7, loss = 1148460
Epoch 0, Iteration 8, loss = 7782816
Epoch 0, Iteration 9, loss = 855826
Epoch 0, Iteration 10, loss = 1072620
Epoch 0, Iteration 11, loss = 1053771
Epoch 0, Iteration 12, loss = 1222515
Epoch 0, Iteration 13, loss = 644118
Epoch 0, Iteration 14, loss = 549055
Epoch 0, Iteration 15, loss = 2336090
Epoch 0, Iteration 16, loss = 798603
Epoch 0, Iteration 17, loss = 1007528
Epoch 0, Iteration 18, loss = 596247
Epoch 0, Iteration 19, loss = 778190
Epoch 0, Iteration 20, loss

In [14]:
# train a baseline model - vgg16
fix_random_seed(0)

C, H, W = 3, 200, 200

# initialize the output layer with 1 node for regression tasks.
output_size = 1 

learning_rate = 1e-4
dropout_ratio = 0.1
epochs = 5

# load the vgg16 model
model = models.vgg16(pretrained=True)

# modify the classifier to fit the guitar image dataset
model.classifier = nn.Sequential(
    nn.Linear(25088, 512),  
    nn.ReLU(),
    nn.Dropout(dropout_ratio),
    nn.Linear(512, 1)  # output a single value for regression
)

model.to(device)

# use adam as the optimizer
optimizer = optim.Adam(model.parameters(), 
                       lr = learning_rate)

# print the Architecture of the model
print('Architecture:')
print(model)
print()

# print the loss in the training process
print('Training:')
train_model(model, optimizer, epochs = epochs)

Architecture:
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding

Epoch 4, Iteration 162, loss = 822997
Epoch 4, Iteration 163, loss = 663321
Epoch 4, Iteration 164, loss = 544015
Epoch 4, Iteration 165, loss = 279096
Epoch 4, Iteration 166, loss = 783173
Epoch 4, Iteration 167, loss = 657318
Epoch 4, Iteration 168, loss = 627608
Epoch 4, Iteration 169, loss = 326122
